In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.cluster import DBSCAN, OPTICS
import itertools

In [2]:
import bs4 as bs
import pickle
import requests

In [3]:
import pandas as pd

# There are 2 tables on the Wikipedia page
# we want the first table

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = payload[0]
second_table = payload[1]

df2 = first_table

In [4]:
df2.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 505 non-null    object
 1   Security               505 non-null    object
 2   SEC filings            505 non-null    object
 3   GICS Sector            505 non-null    object
 4   GICS Sub-Industry      505 non-null    object
 5   Headquarters Location  505 non-null    object
 6   Date first added       457 non-null    object
 7   CIK                    505 non-null    int64 
 8   Founded                505 non-null    object
dtypes: int64(1), object(8)
memory usage: 35.6+ KB


In [6]:
df2

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [7]:
df2 = df2[df2['Date first added']<'2015-01-01']

In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 504
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 320 non-null    object
 1   Security               320 non-null    object
 2   SEC filings            320 non-null    object
 3   GICS Sector            320 non-null    object
 4   GICS Sub-Industry      320 non-null    object
 5   Headquarters Location  320 non-null    object
 6   Date first added       320 non-null    object
 7   CIK                    320 non-null    int64 
 8   Founded                320 non-null    object
dtypes: int64(1), object(8)
memory usage: 25.0+ KB


In [12]:
import yfinance as yf
symbList = df2['Symbol'].values.tolist()
df1 = yf.download(symbList,'2016-1-1','2021-1-1')['Adj Close']
print(df1.head())

[*********************100%***********************]  320 of 320 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
                    A       AAPL       ABBV        ABC        ABT        ACN  \
Date                                                                           
2015-12-31        NaN        NaN        NaN        NaN        NaN        NaN   
2016-01-04  38.763378  24.286833  44.612743  92.183670  38.476147  92.542465   
2016-01-05  38.629997  23.678219  44.426880  93.531982  38.467175  93.024124   
2016-01-06  38.801476  23.214844  44.434616  92.102219  38.144531  92.842361   
2016-01-07  37.153393  22.235069  44.302967  89.161247  37.230354  90.115990   

                 ADBE        ADI        ADM        ADP  ...       WYNN  \
Date                                                    ...              
2015-12-31        NaN        NaN        NaN        NaN  ...        NaN   
2016-01-04

In [13]:
df = df1.copy()

In [14]:
df.head()

,A,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,...,WYNN,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-04,38.763378,24.286833,44.612743,92.183670,38.476147,92.542465,91.970001,48.338818,30.016403,72.699669,...,62.285862,30.300165,41.560146,58.389812,56.609539,33.650471,46.858719,97.487808,23.599607,45.514313
2016-01-05,38.629997,23.678219,44.426880,93.531982,38.467175,93.024124,92.339996,47.983654,30.351414,72.876839,...,63.463280,30.605707,42.177193,58.887341,57.763653,33.641144,46.741917,99.517784,23.343382,46.226826
2016-01-06,38.801476,23.214844,44.434616,92.102219,38.144531,92.842361,91.019997,45.941418,29.614401,71.982155,...,60.175541,30.928234,41.405876,58.397354,56.926907,33.221436,46.410965,99.948708,22.663050,46.236450
2016-01-07,37.153393,22.235069,44.302967,89.161247,37.230354,90.115990,89.110001,44.769344,28.910889,69.785301,...,54.514839,31.047062,39.772518,57.462631,56.426796,32.363384,44.827599,97.650581,21.982716,44.830688


In [15]:

returns = df['2016-12-31':'2020-12-31'].pct_change()#1 year

In [16]:
returns

,A,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,...,WYNN,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,0.013121,-0.001119,0.014100,0.024815,0.007939,0.002404,0.006378,-0.002069,-0.001732,0.001546,...,0.032243,0.004432,-0.007279,-0.011002,0.008188,0.014904,0.003639,0.009194,0.014358,0.009703
2017-01-05,-0.011889,0.005085,0.007584,-0.011576,0.008638,-0.014991,0.016996,-0.014373,-0.007374,-0.005981,...,0.012849,0.000000,-0.012108,-0.014907,-0.013029,-0.009129,0.003310,0.006425,-0.016210,-0.003327
2017-01-06,0.031156,0.011148,0.000313,0.013384,0.027204,0.011392,0.022566,0.003926,-0.022941,0.000679,...,0.010827,0.002941,0.019334,-0.000565,-0.000686,-0.007010,0.012097,0.000095,0.006498,0.003152
2017-01-09,0.003126,0.009160,0.006584,0.008019,-0.000981,-0.011178,0.002493,0.004749,0.000671,-0.006207,...,0.003462,-0.015152,0.000169,-0.016497,0.003088,-0.004236,0.002794,0.019436,-0.010837,-0.002773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,0.000085,0.007712,-0.000194,-0.002777,0.008376,-0.000544,0.005937,0.008728,0.001824,0.007820,...,-0.007892,0.004976,0.000000,-0.004070,0.004069,0.005214,0.007023,0.003429,-0.003907,0.005443
2020-12-28,0.004433,0.035766,0.001840,-0.001753,-0.005168,0.009250,-0.001820,0.001454,0.005259,0.000283,...,0.009441,0.010676,-0.003310,0.003365,0.009456,0.001197,0.020830,-0.002412,0.000461,0.010391
2020-12-29,-0.005092,-0.013315,0.012083,0.003306,0.005010,-0.007393,0.006333,-0.007050,-0.005030,-0.009852,...,-0.010046,0.001531,-0.011447,-0.011260,-0.020264,-0.008568,-0.003006,0.023910,-0.013607,0.004495


In [17]:
limitPer = len(returns) * .50
returns = returns.dropna(thresh=limitPer,axis=1)
returns.replace([np.inf, -np.inf], np.nan)
returns = returns.fillna(method='ffill')
returns = returns[2:]

In [18]:
returns.to_excel('price_matrix.xlsx')

In [ ]:
## The distance approach identifies pairs using the correlation of (normalized) asset prices or their returns and is simple and orders of magnitude less computationally intensive than cointegration tests.
## 

In [19]:
# compute PCA on returns matrix with 90% of variance retained
pca = PCA(n_components=0.9)
pca.fit(returns)
print(pca.components_.T.shape)

(317, 105)


In [20]:
# standardize data
X = preprocessing.StandardScaler().fit_transform(pca.components_.T)

In [21]:
# check results for DBScan
for val in [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1,1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 3]:
    clf = DBSCAN(eps=val, min_samples=2)

    clf.fit(X)
    labels = clf.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    print(f"\nClusters discovered: {n_clusters_} for val {val}")

    clustered = clf.labels_


Clusters discovered: 0 for val 0.01

Clusters discovered: 0 for val 0.02

Clusters discovered: 0 for val 0.05

Clusters discovered: 0 for val 0.1

Clusters discovered: 0 for val 0.2

Clusters discovered: 0 for val 0.5

Clusters discovered: 1 for val 0.6

Clusters discovered: 1 for val 0.7

Clusters discovered: 1 for val 0.8

Clusters discovered: 1 for val 0.9

Clusters discovered: 1 for val 1

Clusters discovered: 1 for val 1.1

Clusters discovered: 1 for val 1.2

Clusters discovered: 1 for val 1.3

Clusters discovered: 1 for val 1.4

Clusters discovered: 1 for val 1.5

Clusters discovered: 2 for val 1.6

Clusters discovered: 2 for val 1.7

Clusters discovered: 2 for val 1.8

Clusters discovered: 2 for val 1.9

Clusters discovered: 2 for val 2

Clusters discovered: 3 for val 3


In [22]:
# fit OPTICS
clustering = OPTICS(min_samples=2).fit(X)


In [23]:
n_clusters_ = len(set(clustering.labels_)) - (1 if -1 in labels else 0)
clustered = clustering.labels_

clustered_series = pd.Series(index=returns.columns, data=clustered.flatten())
clustered_series_all = pd.Series(index=returns.columns, data=clustered.flatten())
clustered_series = clustered_series[clustered_series != -1]

In [24]:
pairs = []

for clust in clustered_series.unique():
    tickers = list(clustered_series[clustered_series==clust].index)
    all_comb = list(itertools.combinations(tickers,2))
    
    #Create all pairs within a cluster with more than 2 stocks
    for i in all_comb:
        pairs.append(i[0] + '_' + i[1])


In [25]:
clustered_pairs = pd.DataFrame(columns = ['Pairs'])
clustered_pairs['Pairs'] = pairs

In [26]:
clustered_pairs.to_excel('Clustered_pairs.xlsx', index = None)

ADF Test for Cointegration

In [27]:
import warnings
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from tqdm import tqdm

def test_cointegration(stocks, possible_pairs, lookback=2):
    test_start = '2016-01-01'
    test_end = '2016-12-31'
    results = []

    
    possible_pairs = pairs
    stock_data = stocks.loc[test_start:test_end]

    for pair in tqdm(possible_pairs):
        s1 = pair.split('_')[0]
        s2 = pair.split('_')[1]
        df = stock_data[[s1, s2]].dropna()
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            var = VAR(df)
            try:
                lags = var.select_order()
                order = lags.selected_orders['aic']
            except:
                #For some stocks price was same throughout the period...for these kind of data, VAR gives error, so order 1 is taken.
                lags = 'NA'
                order = 1
            result = [test_end, s1, s2]

            result += [coint(df[s1], df[s2], trend='c')[1], coint(df[s2], df[s1], trend='c')[1]]
        try:
            cj = coint_johansen(df, det_order=0, k_ar_diff=order)
            #lr1 is the trace statistics, l2 is maximum eigenvalue statistic, cj.evec is the eigenvectors of VECM coeff matrix
            result += (list(cj.lr1) + list(cj.lr2) + list(cj.evec[:, cj.ind[0]]))
        except:
            #Same price throughout the data, for this Johansen Test gives error
            result += (list([0,0]) + list([0,0]) + list([0,0]))

        results.append(result)
    return results





In [28]:
result1 = test_cointegration(df, pairs)

100%|██████████| 77/77 [00:03<00:00, 19.27it/s]


In [29]:
columns = ['test_end', 's1', 's2', 'eg1', 'eg2',
           'trace0', 'trace1', 'eig0', 'eig1', 'w1', 'w2']

critical_values = {0: {.9: 13.4294, .95: 15.4943, .99: 19.9349},
                   1: {.9: 2.7055, .95: 3.8415, .99: 6.6349}}

trace0_cv = critical_values[0][.95] # critical value for 0 cointegration relationships
trace1_cv = critical_values[1][.95] # critical value for 1 cointegration relationship

all_tests = []
all_tests.append(pd.DataFrame(result1, columns=columns))
test_results = pd.concat(all_tests)

# extract results that are significant from Johansen
test_results['joh_sig'] = ((test_results.trace0 > trace0_cv) &
                           (test_results.trace1 > trace1_cv))

test_results['eg'] = test_results[['eg1', 'eg2']].min(axis=1)
test_results['s1_dep'] = test_results.eg1 < test_results.eg2
# extract results that are significant from Englage-Granger...Less than 0.05 in either of eg1 and eg2
test_results['eg_sig'] = (test_results.eg < .05)

# select pairs that are significant from both methods
test_results['coint'] = (test_results.eg_sig & test_results.joh_sig)
test_results.coint.value_counts(normalize=True)

False    0.922078
True     0.077922
Name: coint, dtype: float64

In [30]:
def select_candidate_pairs(data):
    candidates = data[data.joh_sig | data.eg_sig]
    candidates['y'] = candidates.apply(lambda x: x.s1 if x.s1_dep else x.s2, axis=1)
    candidates['x'] = candidates.apply(lambda x: x.s2 if x.s1_dep else x.s1, axis=1)
    return candidates.drop(['s1_dep', 's1', 's2'], axis=1)

candidates = select_candidate_pairs(test_results)

best_candidates = candidates[candidates['coint'] == True]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
best_candidates.to_excel('best_pairs.xlsx', index = None)